<a href="https://colab.research.google.com/github/tylerlum/self_driving_car/blob/master/Behavioral_Cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autonomous Driving

In [1]:
## Import data
!git clone https://github.com/arnold402/data

In [2]:
!ls data

In [3]:
!pip3 install imgaug

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as npimg
import os

## Keras
import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
#import Conv2D

from imgaug import augmenters as iaa
import cv2
import pandas as pd
import random
import ntpath

## Sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Load Data

In [5]:
## Store data
datadir = 'data'
columns = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
data = pd.read_csv(os.path.join(datadir, 'driving_log.csv'), names = columns)
#Show full width of dataframe column.
pd.set_option('display.max_colwidth', -1)
data.head()


In [6]:
def path_leaf(path):
  """Get tail of path
  
  Parameters
  ___________
  
  path: String
      path of an image
      
  Return
  ___________
  tail: String
      only the tail (image name) of an image path
  
  """
  head, tail = ntpath.split(path)
  return tail

## Remove path of images
data['center'] = data['center'].apply(path_leaf)
data['left'] = data['left'].apply(path_leaf)
data['right'] = data['right'].apply(path_leaf)
data.head()

The histogram shows that our data has lot of 0 angles which could bias the model to 0 angle if not corrected. Therefore we would like to remove all the data beyond 200 threshold

In [7]:
## Visualize data
num_bins = 25
samples_per_bin = 200
hist, bins = np.histogram(data['steering'], num_bins)
center = bins[:-1] + bins[1:] * 0.5  # center the bins to 0

## Plot
plt.bar(center, hist, width=0.05)
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin))

Here we correct the bias.

In [8]:
## Unskew data: for bins with >samples_per_bin, shuffle them and remove enough to have samples_per_bin 
print('Total data: {0}'.format(len(data)))

## Make list of indices to remove
remove_list = []
for j in range(num_bins):
  list_ = []
  for i in range(len(data['steering'])):
    steering_angle = data['steering'][i]
    if steering_angle >= bins[j] and steering_angle <= bins[j+1]:
      list_.append(i)
  list_ = shuffle(list_)
  list_ = list_[samples_per_bin:]
  remove_list.extend(list_)
  
## Remove extras from list
data.drop(data.index[remove_list], inplace=True)
print('Removed: {0}'.format(len(remove_list)))
print('Remaining: {0}'.format(len(data)))

## Plot
hist, _ = np.histogram(data['steering'], (num_bins))
plt.bar(center, hist, width=0.05)
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin))

### Load image steering 

In [9]:
def load_img_steering(datadir, df):
  """Get img and steering data into arrays
  
  Parameters
  ___________
  
  datadir: String
      directory of the dataset
  df: Dataframe
    dataframe contain the dataset
      
  Return
  ___________
  image_paths: array
      image paths
  steerings: array
      staring angles
  
  """
  image_path = []
  steering = []
  for i in range(len(data)):
    indexed_data = data.iloc[i]
    center, left, right = indexed_data[0], indexed_data[1], indexed_data[2]
    image_path.append(os.path.join(datadir, center.strip()))
    steering.append(float(indexed_data[3]))
  image_paths = np.asarray(image_path)
  steerings = np.asarray(steering)
  return image_paths, steerings

image_paths, steerings = load_img_steering(datadir + '/IMG', data)
  

# Train Test Split

In [10]:
## Split data into training and validation
X_train, X_valid, Y_train, Y_valid = train_test_split(image_paths, steerings, test_size=0.2, random_state=0)

## Check that data is valid
print("Training Samples: {}\nValid Samples: {}".format(len(X_train), len(X_valid)))
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(Y_train, bins=num_bins, width=0.05, color='blue')
axes[0].set_title('Training set')
axes[1].hist(Y_valid, bins=num_bins, width=0.05, color='red')
axes[1].set_title('Validation set')

# Augmentation

In [11]:
def zoom(image):
  """
  Zoom the image
  
  Parameters
  ___________
  
  image: array
      image as numpy array
      
  Return
  ___________
  image: array
      zoomed image as numpy array
  """  
  zoom = iaa.Affine(scale=(1, 1.3))
  image = zoom.augment_image(image)
  return image

In [12]:
## Test zoom function
image = image_paths[random.randint(0, 1000)]
original_image = npimg.imread(image)
zoomed_image = zoom(original_image)

fix, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()

axs[0].imshow(original_image)
axs[0].set_title('Original Image')
axs[1].imshow(zoomed_image)
axs[1].set_title('Zoomed Image')

In [13]:
def pan(image):
  """
  Pans the image
  
  Parameters
  ___________
  
  image: array
      image as numpy array
      
  Return
  ___________
  image: array
      panned image as numpy array
  """ 
  pan = iaa.Affine(translate_percent= {'x': (-0.1, 0.1), 'y': (-0.1, 0.1)})
  image = pan.augment_image(image)
  return image

In [14]:
## Test pan function
image = image_paths[random.randint(0, 1000)]
original_image = npimg.imread(image)
panned_image = pan(original_image)

fix, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()

axs[0].imshow(original_image)
axs[0].set_title('Original Image')
axs[1].imshow(panned_image)
axs[1].set_title('Panned Image')

In [15]:
def img_random_brightness(image):
  """
  Randomly adjust image brightness
  
  Parameters
  ___________
  
  image: array
      image as numpy array
      
  Return
  ___________
  image: array
      image as numpy array
  """ 
  brightness = iaa.Multiply((0.2, 1.2))
  image = brightness.augment_image(image)
  return image

In [16]:
## Test random brightness function
image = image_paths[random.randint(0, 1000)]
original_image = npimg.imread(image)
brightness_altered_image = img_random_brightness(original_image)

fix, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()

axs[0].imshow(original_image)
axs[0].set_title('Original Image')
axs[1].imshow(brightness_altered_image)
axs[1].set_title('Brightness Altered Image')

In [17]:
def img_random_flip(image, steering_angle):
  """
  Horizontally flips the image
  
  Parameters
  ___________
  
  image: array
      image as numpy array
      
  Return
  ___________
  image: array
      fliped image as numpy array
  """ 
  image = cv2.flip(image, 1)  # horizontal flip
  steering_angle = -steering_angle  # must flip steering angle
  return image, steering_angle

In [18]:
## Test flip function
random_index = random.randint(0, 1000)
image = image_paths[random_index]
steering_angle = steerings[random_index]

original_image = npimg.imread(image)
flipped_image, flipped_steering_angle = img_random_flip(original_image, steering_angle)

fix, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()

axs[0].imshow(original_image)
axs[0].set_title('Original Image - {}'.format(steering_angle))
axs[1].imshow(flipped_image)
axs[1].set_title('Flipped Image - {}'.format(flipped_steering_angle))

In [19]:
def random_augment(image, steering_angle):
  """Augment with 50% change for each augmentation technique
  Parameters
  ___________
  
  image: array
      image as numpy array
  steering_angle: array
      steering angle
      
  Return
  ___________
  image: array
      augmented image as numpy array
  
  """
  image = npimg.imread(image)
  if np.random.rand() < 0.5:
    image = pan(image)
  if np.random.rand() < 0.5:
    image = zoom(image)
  if np.random.rand() < 0.5:
    image = img_random_brightness(image)
  if np.random.rand() < 0.5:
    image, steering_angle = img_random_flip(image, steering_angle)
  return image, steering_angle

In [20]:
#Augment the images
ncol = 2
nrow = 10
fig, axs = plt.subplots(nrow, ncol, figsize=(15, 50))
fig.tight_layout()

for i in range(nrow):
  randnum = random.randint(0, len(image_paths)-1)
  random_image = image_paths[randnum]
  random_steering = steerings[randnum]
  
  original_image = npimg.imread(random_image)
  augmented_image, steering = random_augment(random_image, random_steering)
  
  axs[i][0].imshow(original_image)
  axs[i][0].set_title('Original Image - {}'.format(random_steering))
  axs[i][1].imshow(augmented_image)
  axs[i][1].set_title('Augmented Image - {}'.format(steering))

# Image Preprocessing

In [21]:
def img_path_preprocess(img_path):
  """Take in img, returns preprocessed image
  
  Parameters
  ___________
  
  image_path: string
      image path
      
  Return
  ___________
  image: array
      preprocessed images
  """
  ## Crop image to remove unnecessary features
  img = npimg.imread(img_path)
  return img_preprocess(img)

def img_preprocess(img):
  img = img[60:135, :, :]
  
  ## Change to YUV image
  img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
  
  ## Gaussian blur
  img = cv2.GaussianBlur(img, (3, 3), 0)
  
  ## Decrease size for easier processing
  img = cv2.resize(img, (200, 66))
  
  ## Normalize values
  img = img / 255
  return img

In [22]:
## Get any image
image = image_paths[100]
original_image = npimg.imread(image)
preprocessed_image = img_path_preprocess(image)

## Compare original and preprocessed image
fig, axes = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axes[0].imshow(original_image)
axes[0].set_title('Original Image')
axes[1].imshow(preprocessed_image)
axes[1].set_title('Preprocessed Image')

In [23]:
def batch_generator(image_paths, steering_angle, batch_size, is_training):
  """Generate small batches of images to avoid memory issues"""
  while True:
    batch_img = []
    batch_steering = []
    
    for i in range(batch_size):
      random_index = random.randint(0, len(image_paths)-1)
      
      if is_training:
        im, steering = random_augment(image_paths[random_index], steering_angle[random_index])
      else:
        im = npimg.imread(image_paths[random_index])
        steering = steering_angle[random_index]
      
      im = img_preprocess(im)
      batch_img.append(im)
      batch_steering.append(steering)
      
    yield(np.asarray(batch_img), np.asarray(batch_steering))

In [24]:
X_train_gen, Y_train_gen = next(batch_generator(X_train, Y_train, 1, True))
X_valid_gen, Y_valid_gen = next(batch_generator(X_valid, Y_valid, 1, False))

fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()

axs[0].imshow(X_train_gen[0])
axs[0].set_title('Training Image')

axs[1].imshow(X_valid_gen[0])
axs[1].set_title('Validation Image')

In [25]:
## Preprocess data
X_train = np.array(list(map(img_path_preprocess, X_train)))
X_valid = np.array(list(map(img_path_preprocess, X_valid)))

In [26]:
plt.imshow(X_train[random.randint(0, len(X_train)-1)])
plt.axis('off')
print(X_train.shape)

# Model Architecture

In [27]:
def nvidia_model():
  """Create Model architecture"""
  model = Sequential()
  model.add(Conv2D(24, 5, 2, input_shape = (66, 200, 3), activation = 'relu'))
  model.add(Conv2D(36, 5, 2, activation = 'relu'))
  model.add(Conv2D(48, 5, 2, activation = 'relu'))
  model.add(Conv2D(64, 3, activation = 'relu'))
  model.add(Conv2D(64, 3, activation = 'relu'))
  model.add(Dropout(0.5))
  
  model.add(Flatten())
  
  model.add(Dense(100, activation='elu'))
  model.add(Dropout(0.5))
  
  model.add(Dense(50, activation='elu'))
  model.add(Dropout(0.5))
  
  model.add(Dense(10, activation='elu'))
  model.add(Dropout(0.5))
  
  model.add(Dense(1))
  
  optimizer = Adam(learning_rate=1e-3)
  model.compile(loss='mse', optimizer=optimizer)
  return model

# Model Summary

In [28]:
model = nvidia_model()
print(model.summary())

# Fit the Model

In [29]:
history = model.fit(X_train, Y_train, epochs=40, validation_data=(X_valid, Y_valid), batch_size=100, verbose=1, shuffle=1)

In [30]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'])
plt.title('Loss')
plt.xlabel('Epoch')

# Save

In [31]:
model.save('model.h5')